### Ok this is my fourth attempt.
###### This time I will split each match into two rows, one for the winner, one for the loser. 
###### Different POVs, and an indicator of the victor

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

atp_2021_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2021.csv', parse_dates=[5])
atp_2022_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2022.csv', parse_dates=[5])

atp_2023_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2023.csv', parse_dates=[5])

In [2]:
atp_frame = pd.concat([atp_2021_frame, atp_2022_frame, atp_2023_frame], axis = 0)

In [3]:
atp_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8636 entries, 0 to 2985
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tourney_id          8636 non-null   object        
 1   tourney_name        8636 non-null   object        
 2   surface             8583 non-null   object        
 3   draw_size           8636 non-null   int64         
 4   tourney_level       8636 non-null   object        
 5   tourney_date        8636 non-null   datetime64[ns]
 6   match_num           8636 non-null   int64         
 7   winner_id           8636 non-null   int64         
 8   winner_seed         3635 non-null   float64       
 9   winner_entry        1318 non-null   object        
 10  winner_name         8636 non-null   object        
 11  winner_hand         8634 non-null   object        
 12  winner_ht           8354 non-null   float64       
 13  winner_ioc          8636 non-null   object      

In [4]:
#### drop rows containing certain null values
atp_frame = atp_frame.dropna(subset = ['winner_age', 'loser_age','winner_rank','loser_rank', 'winner_hand', 'loser_hand', 'winner_ht', 'loser_ht'])

In [5]:
atp_frame.iloc[55:58]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
55,2021-0096,Tokyo Olympics,Hard,64,A,2021-07-24,292,104925,1.0,NaN,Novak Djokovic,R,188.0,SRB,34.1,200221,16.0,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,22.1,6-3 6-1,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12113.0,35.0,1723.0
56,2021-0096,Tokyo Olympics,Hard,64,A,2021-07-24,293,111575,12.0,NaN,Karen Khachanov,R,198.0,RUS,25.1,200005,14.0,NaN,Ugo Humbert,L,188.0,FRA,23.0,7-6(4) 4-6 6-3,3,QF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,2235.0,28.0,2090.0
57,2021-0096,Tokyo Olympics,Hard,64,A,2021-07-24,294,100644,4.0,NaN,Alexander Zverev,R,198.0,GER,24.2,104871,NaN,NaN,Jeremy Chardy,R,188.0,FRA,34.4,6-4 6-1,3,QF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7340.0,68.0,1019.0


In [6]:

atp_frame['surface_cat'] = atp_frame['surface'].astype("category").cat.codes ## Add a surface category column

atp_frame['winner_hand'] = atp_frame['winner_hand'].astype("category").cat.codes ## convert hand to an int
atp_frame['loser_hand'] = atp_frame['loser_hand'].astype("category").cat.codes ## convert hand to an int    


In [7]:
atp_frame.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'surface_cat'],
      dtype='object')

In [8]:
### make df for just winners
#columns_to_keep_for_winners = ['tourney_name','surface','match_num','winner_id','winner_name','winner_hand','winner_ht','winner_age', 'minutes',
 #                              'w_ace', 'w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'winner_rank']




winning_rename_lib = {'winner_id':'player_id', 'winner_seed':'player_seed', 'winner_entry':'player_entry',
       'winner_name':'player_name', 'winner_hand':'player_hand', 'winner_ht':'player_ht', 'winner_ioc':'player_ioc', 'winner_age':'player_age',
       'loser_id':'opponent_id', 'loser_seed':'opponent_seed', 'loser_entry':'opponent_entry', 'loser_name':'opponent_name', 'loser_hand':'opponent_hand',
       'loser_ht': 'opponent_ht', 'loser_ioc':'opponent_ioc', 'loser_age':'opponent_age',
        'w_ace':'player_ace', 'w_df':'player_df', 'w_svpt':'player_svpt', 'w_1stIn':'player_1stIn', 'w_1stWon':'player_1stWon', 'w_2ndWon':'player_2ndWon',
       'w_SvGms':'player_SvGms', 'w_bpSaved':'player_bpSaved', 'w_bpFaced':'player_bpFaced', 'l_ace':'opponent_ace', 'l_df':'opponent_df', 'l_svpt':'opponent_svpt',
       'l_1stIn':'opponent_1stIn', 'l_1stWon':'opponent_1stWon', 'l_2ndWon':'opponent_2ndWon', 'l_SvGms':'opponent_SvGms', 'l_bpSaved':'opponent_bpSaved', 'l_bpFaced':'opponent_bpFaced',
       'winner_rank':'player_rank', 'winner_rank_points':'player_rank_points', 'loser_rank':'opponent_rank', 'loser_rank_points':'opponent_rank_points'}

losing_rename_lib = {'winner_id':'opponent_id', 'winner_seed':'opponent_seed', 'winner_entry':'opponent_entry',
       'winner_name':'opponent_name', 'winner_hand':'opponent_hand', 'winner_ht':'opponent_ht', 'winner_ioc':'opponent_ioc', 'winner_age':'opponent_age',
       'loser_id':'player_id', 'loser_seed':'player_seed', 'loser_entry':'player_entry', 'loser_name':'player_name', 'loser_hand':'player_hand',
       'loser_ht': 'player_ht', 'loser_ioc':'player_ioc', 'loser_age':'player_age',
        'w_ace':'opponent_ace', 'w_df':'opponent_df', 'w_svpt':'opponent_svpt', 'w_1stIn':'opponent_1stIn', 'w_1stWon':'opponent_1stWon', 'w_2ndWon':'opponent_2ndWon',
       'w_SvGms':'opponent_SvGms', 'w_bpSaved':'opponent_bpSaved', 'w_bpFaced':'opponent_bpFaced', 'l_ace':'player_ace', 'l_df':'player_df', 'l_svpt':'player_svpt',
       'l_1stIn':'player_1stIn', 'l_1stWon':'player_1stWon', 'l_2ndWon':'player_2ndWon', 'l_SvGms':'player_SvGms', 'l_bpSaved':'player_bpSaved', 'l_bpFaced':'player_bpFaced',
       'winner_rank':'opponent_rank', 'winner_rank_points':'opponent_rank_points', 'loser_rank':'player_rank', 'loser_rank_points':'player_rank_points'}

#losing_rename_lib = {'loser_rank':'player_rank', 'winner_rank':'opponent_rank', 'loser_age':'player_age', 'winner_age':'opponent_age', 'loser_ht':'player_height', 'winner_ht':'opponent_height', 'loser_hand':'player_hand', 'winner_hand':'opponent_hand'} #rename columns
    
winners_df = atp_frame.rename(columns = winning_rename_lib)
losers_df = atp_frame.rename(columns = losing_rename_lib)

#winners_of_2022_df = atp_2022_frame[columns_to_keep_for_winners]

losers_df['result'] = 0
winners_df['result'] = 1

each_match_twice = pd.concat([winners_df, losers_df], axis = 0)

#each_match_twice_2022 = winners_of_2022_df.append(losers_of_2022_df)#pd.concat([winners_of_2022_df, losers_of_2022_df])

In [9]:
 ### trying to make sure each player's matches are in consecutive order (i think im satisfied with the result)
each_match_twice = each_match_twice.sort_values(by = ['tourney_date', 'tourney_id','match_num'])

In [10]:
## Setting up 'convenience columns' to help calculate other things

each_match_twice['player_holds'] = each_match_twice['player_SvGms'] - (each_match_twice['player_bpFaced'] - each_match_twice['player_bpSaved'])
each_match_twice['player_breaks'] = each_match_twice['opponent_bpFaced'] - each_match_twice['opponent_bpSaved']
each_match_twice['opponent_holds'] = each_match_twice['opponent_SvGms'] - (each_match_twice['opponent_bpFaced'] - each_match_twice['opponent_bpSaved'])
each_match_twice['opponent_breaks'] = each_match_twice['player_bpFaced'] - each_match_twice['player_bpSaved']

In [11]:
#### make some columns for things like break point coverted percent, ace to df percent, bp saved percent .... etcetera

each_match_twice['player_break_pct'] = each_match_twice['player_breaks'] / each_match_twice['opponent_SvGms']
each_match_twice['player_hold_pct'] = (each_match_twice['player_SvGms'] - each_match_twice['opponent_breaks']) / each_match_twice['player_SvGms']
each_match_twice['player_games_won_pct'] = (each_match_twice['player_holds'] + each_match_twice['player_breaks']) / (each_match_twice['player_SvGms'] + each_match_twice['opponent_SvGms'])

In [12]:
each_match_twice.iloc[350:359]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,score,best_of,round,minutes,player_ace,player_df,player_svpt,player_1stIn,player_1stWon,player_2ndWon,player_SvGms,player_bpSaved,player_bpFaced,opponent_ace,opponent_df,opponent_svpt,opponent_1stIn,opponent_1stWon,opponent_2ndWon,opponent_SvGms,opponent_bpSaved,opponent_bpFaced,player_rank,player_rank_points,opponent_rank,opponent_rank_points,surface_cat,result,player_holds,player_breaks,opponent_holds,opponent_breaks,player_break_pct,player_hold_pct,player_games_won_pct
2165,2021-8888,Atp Cup,Hard,12,A,2021-02-02,279,103917,NaN,NaN,Nicolas Mahut,1,190.0,FRA,39.0,110602,NaN,NaN,Dennis Novak,1,183.0,AUT,27.4,7-6(2) 6-2,3,RR,87.0,12.0,1.0,64.0,43.0,36.0,11.0,10.0,3.0,4.0,4.0,2.0,67.0,44.0,25.0,12.0,10.0,2.0,5.0,239.0,239.0,100.0,737.0,2,1,9.0,3.0,7.0,1.0,0.300000,0.900000,0.600000
2165,2021-8888,Atp Cup,Hard,12,A,2021-02-02,279,110602,NaN,NaN,Dennis Novak,1,183.0,AUT,27.4,103917,NaN,NaN,Nicolas Mahut,1,190.0,FRA,39.0,7-6(2) 6-2,3,RR,87.0,4.0,2.0,67.0,44.0,25.0,12.0,10.0,2.0,5.0,12.0,1.0,64.0,43.0,36.0,11.0,10.0,3.0,4.0,100.0,737.0,239.0,239.0,2,0,7.0,1.0,9.0,3.0,0.100000,0.700000,0.400000
2166,2021-8888,Atp Cup,Hard,12,A,2021-02-02,280,106233,NaN,NaN,Dominic Thiem,1,185.0,AUT,27.4,105332,NaN,NaN,Benoit Paire,1,196.0,FRA,31.7,6-1 0-0 RET,3,RR,22.0,5.0,0.0,17.0,10.0,10.0,6.0,4.0,0.0,0.0,0.0,5.0,24.0,16.0,11.0,0.0,3.0,2.0,4.0,3.0,9125.0,28.0,1738.0,2,1,4.0,2.0,1.0,0.0,0.666667,1.000000,0.857143
2166,2021-8888,Atp Cup,Hard,12,A,2021-02-02,280,105332,NaN,NaN,Benoit Paire,1,196.0,FRA,31.7,106233,NaN,NaN,Dominic Thiem,1,185.0,AUT,27.4,6-1 0-0 RET,3,RR,22.0,0.0,5.0,24.0,16.0,11.0,0.0,3.0,2.0,4.0,5.0,0.0,17.0,10.0,10.0,6.0,4.0,0.0,0.0,28.0,1738.0,3.0,9125.0,2,0,1.0,0.0,4.0,2.0,0.000000,0.333333,0.142857
2167,2021-8888,Atp Cup,Hard,12,A,2021-02-02,281,110602,NaN,NaN,Dennis Novak,1,183.0,AUT,27.4,104926,NaN,NaN,Fabio Fognini,1,178.0,ITA,33.6,6-3 6-2,3,RR,72.0,2.0,1.0,55.0,39.0,27.0,10.0,9.0,0.0,1.0,3.0,6.0,61.0,39.0,24.0,6.0,8.0,6.0,10.0,100.0,737.0,17.0,2420.0,2,1,8.0,4.0,4.0,1.0,0.500000,0.888889,0.705882
2167,2021-8888,Atp Cup,Hard,12,A,2021-02-02,281,104926,NaN,NaN,Fabio Fognini,1,178.0,ITA,33.6,110602,NaN,NaN,Dennis Novak,1,183.0,AUT,27.4,6-3 6-2,3,RR,72.0,3.0,6.0,61.0,39.0,24.0,6.0,8.0,6.0,10.0,2.0,1.0,55.0,39.0,27.0,10.0,9.0,0.0,1.0,17.0,2420.0,100.0,737.0,2,0,4.0,1.0,8.0,4.0,0.111111,0.500000,0.294118
2168,2021-8888,Atp Cup,Hard,12,A,2021-02-02,282,126610,NaN,NaN,Matteo Berrettini,1,196.0,ITA,24.8,106233,NaN,NaN,Dominic Thiem,1,185.0,AUT,27.4,6-2 6-4,3,RR,81.0,5.0,0.0,50.0,34.0,24.0,10.0,9.0,0.0,1.0,2.0,0.0,61.0,37.0,23.0,10.0,9.0,5.0,9.0,10.0,3120.0,3.0,9125.0,2,1,8.0,4.0,5.0,1.0,0.444444,0.888889,0.666667
2168,2021-8888,Atp Cup,Hard,12,A,2021-02-02,282,106233,NaN,NaN,Dominic Thiem,1,185.0,AUT,27.4,126610,NaN,NaN,Matteo Berrettini,1,196.0,ITA,24.8,6-2 6-4,3,RR,81.0,2.0,0.0,61.0,37.0,23.0,10.0,9.0,5.0,9.0,5.0,0.0,50.0,34.0,24.0,10.0,9.0,0.0,1.0,3.0,9125.0,10.0,3120.0,2,0,5.0,1.0,8.0,4.0,0.111111,0.555556,0.333333
2169,2021-8888,Atp Cup,Hard,12,A,2021-02-02,283,105357,NaN,NaN,John Millman,1,183.0,AUS,31.6,133018,NaN,NaN,Michail Pervolarakis,1,193.0,CYP,24.6,6-2 6-3,3,RR,81.0,5.0,1.0,50.0,29.0,22.0,17.0,9.0,4.0,4.0,1.0,2.0,56.0,28.0,18.0,13.0,8.0,5.0,8.0,38.0,1421.0,462.0,80.0,2,1,9.0,3.0,5.0,0.0,0.375000,1.000000,0.705882


In [13]:
grouped_matches = each_match_twice.groupby('player_name')

In [14]:
single_player = grouped_matches.get_group('John Millman')

#single_player

In [15]:
def rolling_averages(group, columns, new_columns):
    rolling_stats = group[columns].rolling(5, closed='left').mean()
    group[new_columns] = rolling_stats
    group = group.dropna(subset=new_columns)
    return group

In [16]:
columns = ['player_df', 'player_1stIn','player_1stWon', 'player_2ndWon', 'result', 'player_break_pct', 'player_hold_pct', 'player_games_won_pct']
new_columns = [f"{c}_rolling" for c in columns]

In [17]:
rolling_averages(single_player, columns, new_columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_40260\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group[new_columns] = rolling_stats
C:\Users\aaron\AppData\Local\Temp\ipykernel_40260\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group[new_columns] = rolling_stats
C:\Users\aaron\AppData\Local\Temp\ipykernel_40260\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,score,best_of,round,minutes,player_ace,player_df,player_svpt,player_1stIn,player_1stWon,player_2ndWon,player_SvGms,player_bpSaved,player_bpFaced,opponent_ace,opponent_df,opponent_svpt,opponent_1stIn,opponent_1stWon,opponent_2ndWon,opponent_SvGms,opponent_bpSaved,opponent_bpFaced,player_rank,player_rank_points,opponent_rank,opponent_rank_points,surface_cat,result,player_holds,player_breaks,opponent_holds,opponent_breaks,player_break_pct,player_hold_pct,player_games_won_pct,player_df_rolling,player_1stIn_rolling,player_1stWon_rolling,player_2ndWon_rolling,result_rolling,player_break_pct_rolling,player_hold_pct_rolling,player_games_won_pct_rolling
1060,2021-0451,Doha,Hard,32,A,2021-03-08,272,105357,NaN,NaN,John Millman,1,183.0,AUS,31.7,105932,NaN,NaN,Nikoloz Basilashvili,1,185.0,GEO,29.0,2-6 6-4 6-2,3,R32,112.0,3.0,0.0,79.0,48.0,30.0,12.0,13.0,3.0,9.0,7.0,3.0,69.0,42.0,30.0,11.0,13.0,2.0,7.0,41.0,1421.0,42.0,1405.0,2,0,7.0,5.0,8.0,6.0,0.384615,0.538462,0.461538,1.0,46.8,29.6,14.6,0.2,0.180556,0.630556,0.412990
1091,2021-0495,Dubai,Hard,64,A,2021-03-15,252,105357,NaN,NaN,John Millman,1,183.0,AUS,31.7,126523,NaN,Q,Bernabe Zapata Miralles,1,185.0,ESP,24.1,6-4 6-4,3,R64,80.0,3.0,1.0,55.0,36.0,26.0,8.0,10.0,4.0,7.0,1.0,0.0,59.0,38.0,31.0,10.0,10.0,1.0,2.0,40.0,1426.0,150.0,489.0,2,0,7.0,1.0,9.0,3.0,0.100000,0.700000,0.400000,0.8,50.6,31.2,13.6,0.0,0.182479,0.538248,0.364121
2499,2021-9481,Cagliari,Clay,32,A,2021-04-05,282,105357,6.0,NaN,John Millman,1,183.0,AUS,31.8,105948,NaN,NaN,Federico Coria,1,180.0,ARG,29.0,7-6(5) 3-6 6-3,3,R32,193.0,2.0,6.0,108.0,63.0,47.0,20.0,15.0,10.0,14.0,0.0,8.0,127.0,82.0,50.0,22.0,15.0,7.0,11.0,44.0,1436.0,86.0,878.0,0,1,11.0,4.0,11.0,4.0,0.266667,0.733333,0.500000,1.0,49.4,31.6,14.0,0.0,0.180256,0.567137,0.377455
2508,2021-9481,Cagliari,Clay,32,A,2021-04-05,292,105357,6.0,NaN,John Millman,1,183.0,AUS,31.8,111513,NaN,NaN,Laslo Djere,1,188.0,SRB,25.8,6-3 6-3,3,R16,85.0,0.0,2.0,48.0,26.0,15.0,11.0,9.0,3.0,7.0,0.0,1.0,51.0,25.0,18.0,17.0,9.0,2.0,3.0,44.0,1436.0,57.0,1133.0,0,0,5.0,1.0,8.0,4.0,0.111111,0.555556,0.333333,2.2,42.4,28.2,12.6,0.2,0.200256,0.572137,0.389955
661,2021-0410,Monte Carlo Masters,Clay,64,M,2021-04-12,259,105357,NaN,NaN,John Millman,1,183.0,AUS,31.8,200005,NaN,NaN,Ugo Humbert,0,188.0,FRA,22.7,6-3 6-3,3,R64,88.0,1.0,1.0,52.0,33.0,25.0,12.0,9.0,2.0,3.0,0.0,1.0,51.0,26.0,17.0,11.0,9.0,3.0,7.0,44.0,1436.0,32.0,1825.0,0,1,8.0,4.0,5.0,1.0,0.444444,0.888889,0.666667,2.0,40.0,26.2,13.2,0.2,0.222479,0.572137,0.397798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,2022-560,Us Open,Hard,128,G,2022-08-29,121,105357,NaN,NaN,John Millman,1,183.0,AUS,33.2,207182,NaN,NaN,Emilio Nava,1,183.0,USA,20.7,7-6(7) 4-6 7-6(4) 1-6 6-1,5,R128,238.0,11.0,7.0,185.0,111.0,75.0,42.0,26.0,11.0,15.0,15.0,10.0,155.0,95.0,68.0,33.0,24.0,12.0,17.0,103.0,526.0,203.0,258.0,2,0,22.0,5.0,19.0,4.0,0.208333,0.846154,0.540000,1.6,51.8,34.8,18.0,0.4,0.191667,0.809318,0.504200
152,2023-8998,Adelaide 2,Hard,32,A,2023-01-09,279,105357,NaN,Q,John Millman,1,183.0,AUS,33.5,105077,NaN,NaN,Albert Ramos,0,188.0,ESP,34.9,4-6 6-3 7-6(6),3,R32,166.0,7.0,1.0,109.0,77.0,53.0,15.0,16.0,3.0,6.0,7.0,0.0,93.0,68.0,43.0,18.0,15.0,6.0,9.0,148.0,378.0,38.0,1075.0,2,1,13.0,3.0,12.0,3.0,0.200000,0.812500,0.516129,2.6,59.8,40.4,21.8,0.4,0.200000,0.834105,0.523311
143,2023-8998,Adelaide 2,Hard,32,A,2023-01-09,290,105357,NaN,Q,John Millman,1,183.0,AUS,33.5,200221,7.0,NaN,Alejandro Davidovich Fokina,1,183.0,ESP,23.5,6-3 6-3,3,R16,74.0,1.0,4.0,52.0,29.0,18.0,10.0,

In [18]:
## apply the #rolling function to each 'player' group in the dataframe
matches_rolling = each_match_twice.groupby("player_name").apply(lambda x: rolling_averages(x, columns, new_columns))

In [19]:
## This is kinda like reset_index(), it gets rid of the left column
matches_rolling = matches_rolling.droplevel('player_name')

In [20]:
### adding an index by shape
matches_rolling.index = range(matches_rolling.shape[0])
#matches_rolling_2023.index = range(matches_rolling_2023.shape[0])

In [21]:
random_forest = RandomForestClassifier(n_estimators=50, min_samples_split=20, random_state=1, verbose = 1)

In [22]:
predictors = ["player_rank", "opponent_rank", "player_age", "opponent_age",
              "player_ht", "opponent_ht", "surface_cat", "player_hand", "opponent_hand", 
              "player_df_rolling", "player_1stIn_rolling", "player_1stWon_rolling", "player_2ndWon_rolling", "result_rolling", 
              "player_break_pct_rolling", "player_hold_pct_rolling", "player_games_won_pct_rolling"]

simple_preds = ["player_age", "opponent_age",
                "player_ht", "opponent_ht", "player_hand", "opponent_hand", "surface_cat",
               "player_df_rolling", "player_1stIn_rolling", "player_1stWon_rolling", "player_2ndWon_rolling"]

super_simple_preds = ["player_rank", "opponent_rank"]

In [23]:
#### after combining earlier, lets now split into 2022 and 2023 data.
matches_rolling_2022 = matches_rolling[matches_rolling['tourney_date'] < '2022-12-30']
matches_rolling_2023 = matches_rolling[matches_rolling['tourney_date'] > '2023-1-01']

In [24]:
random_forest.fit(matches_rolling_2022[predictors], matches_rolling_2022['result'])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.8s


RandomForestClassifier(min_samples_split=20, n_estimators=50, random_state=1,
                       verbose=1)

In [25]:
predictions = random_forest.predict(matches_rolling_2023[predictors])

predictions.shape

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


(4251,)

In [26]:
accuracy = accuracy_score(matches_rolling_2023['result'], predictions)

accuracy

0.6316160903316866

In [27]:
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int64), array([2005, 2246], dtype=int64))